### SEALION-v3-8B-Instruct LLM run on Aristotle pipeline
### This is run in Runpod/remote Jupyter environment

In [ ]:
!apt update

In [ ]:
!apt install git-lfs

In [ ]:
!pip install transformers safetensors sentencepiece huggingface-hub accelerate bitsandbytes tqdm openai backoff retrying protobuf

In [ ]:
!git lfs install

In [ ]:
!git clone https://huggingface.co/aisingapore/Llama-SEA-LION-v3-8B-IT LLM_MODELS/Llama-SEA-LION-v3-8B-IT

In [ ]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
######### Also useful to reduce thread contention:
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"


snapshot_path = "/workspace/LLM_MODELS/Llama-SEA-LION-v3-8B-IT"
#snapshot_path = "aisingapore/Llama-SEA-LION-v3-8B-IT"

os.environ["LOCAL_MODEL_PATH"] = snapshot_path
os.environ["LLM_MODEL"] = snapshot_path

######## enable 4-bit for quants (and bitsandbytes is set up)
os.environ["LLM_LOAD_IN_4BIT"] = "1"  # or "0" to disable quantization
print("LOCAL_MODEL_PATH =", os.environ["LOCAL_MODEL_PATH"])
print("LLM_MODEL =", os.environ["LLM_MODEL"])

### If kernel doesnt recognize
LLM_MODEL=snapshot_path
LOCAL_MODEL_PATH=snapshot_path


LOCAL_MODEL_PATH = /workspace/LLM_MODELS/Llama-SEA-LION-v3-8B-IT
LLM_MODEL = /workspace/LLM_MODELS/Llama-SEA-LION-v3-8B-IT


In [ ]:
#Test for Max Time generation stopping criteria
from llm_backends import HFBackend
hb = HFBackend(local_model_path=snapshot_path, quantize_4bit=True)
print("Calling short test (3s max_time)...")
res = hb.generate("Write a long list of words and sentences: ", max_new_tokens=1024, max_time=3.0)
print("Result length:", len(res))
print(res[:1000])

print("Calling short test (10s max_time)...")
res = hb.generate("Write a long list of words and sentences: ", max_new_tokens=1024, max_time=10.0)
print("Result length:", len(res))
print(res[:1000])

In [ ]:
# Change this every process (translate, decompose, search_resolve), different time limit may be needed (in seconds).
os.environ["LLM_WORKER_MAX_TIME"] = "300"
LLM_WORKER_MAX_TIME=300

# Commandline args universal
# MAX_NEW_TOKENS is purely for text generation count limit while max_position_embeddings is for context_length based on LLM config.json. !!! input_length + MAX_NEW_TOKENS shopuld be < context_length, otherwise LLM breaks. Llama 3 only has 8k context length/max_posiiton_embedding. SEALIONv3-LLama3-8B-IT uses ROPE, max_position_embeddings follows ROPE limit 131k, Qwen2.5-7B-IT has 32k context length, SahabatAIv1-LLama3-8B-IT has 8k context length.
# Counted the response for each steps in notebook output cell with tokens counter online, translations ~400 tokens, ~decomposition ~500 tokens, search_resolve ~700 tokens
os.environ["MAX_NEW_TOKENS"] = "1500"
MAX_NEW_TOKENS=1500
os.environ["BATCH_NUM"] = "1"
BATCH_NUM=1

In [ ]:
# Translation with original prompts
#!python translate_to_fol.py --data_path manual_data_translated --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file translation --split dev --save_path results_translated_translation/v3/prompts_original --model_name $LLM_MODEL --batch_num $BATCH_NUM --max_new_tokens $MAX_NEW_TOKENS

In [ ]:
# Translation with modified prompts
#!python translate_to_fol.py --data_path manual_data_translated --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file translation_modified --split dev --save_path results_translated_translation/v3/prompts_modified --model_name $LLM_MODEL --batch_num $BATCH_NUM --max_new_tokens $MAX_NEW_TOKENS

In [ ]:
# Translation with refined prompts
#!python translate_to_fol.py --data_path manual_data_translated --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file translation_refine --split dev --save_path results_translated_translation/v3/prompts_refine --model_name $LLM_MODEL --batch_num $BATCH_NUM --max_new_tokens $MAX_NEW_TOKENS

In [ ]:
# Decomposition with refined prompts
#!python decompose_to_cnf.py --data_path results_translated_translation/v3/prompts_refine --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file and_or_decomposer_refine --save_path results_translated_decomposition/v2/prompts_refine --model_name $LLM_MODEL --batch_num $BATCH_NUM --max_new_tokens $MAX_NEW_TOKENS

In [ ]:
#!python negate.py --dataset_name ProntoQA --save_path results_translated_decomposition/v2/prompts_refine --model_name $LLM_MODEL

In [ ]:
# Logic Resolver with original prompts
#!python search_resolve.py --data_path results_translated_decomposition/v2/prompts_refine --dataset_name ProntoQA --sample_pct 0 --prompts_folder manual_prompts_translated --prompts_file logic_resolver  --save_path results_translated_search_resolve/prompts_original --model_name $LLM_MODEL --batch_num $BATCH_NUM --negation False --search_round 10 --max_new_tokens $MAX_NEW_TOKENS 

In [ ]:
#!python search_resolve.py --data_path results_translated_decomposition/v2/prompts_refine --dataset_name ProntoQA --sample_pct 0 --prompts_folder manual_prompts_translated --prompts_file logic_resolver --save_path results_translated_search_resolve/prompts_original --model_name $LLM_MODEL --batch_num $BATCH_NUM --negation True --search_round 10 --max_new_tokens $MAX_NEW_TOKENS

In [ ]:
#!python evaluate.py --dataset_name ProntoQA --save_path results_translated_search_resolve/v2/prompts_original --model_name $LLM_MODEL

ID: ProntoQA_58, GT: B, Ans1: C, Ans2: C
ID: ProntoQA_141, GT: B, Ans1: C, Ans2: B
ID: ProntoQA_380, GT: A, Ans1: C, Ans2: C
ID: ProntoQA_328, GT: A, Ans1: A, Ans2: C
ID: ProntoQA_13, GT: A, Ans1: C, Ans2: C
Total instances: 5
Accuracy: 40.00%
Error id:  ['ProntoQA_58', 'ProntoQA_380', 'ProntoQA_13']
Correct id:  ['ProntoQA_141', 'ProntoQA_328']


In [ ]:
# Logic Resolver with modified prompts
#!python search_resolve.py --data_path results_translated_decomposition/v2/prompts_refine --dataset_name ProntoQA --sample_pct 10 --prompts_folder manual_prompts_translated --prompts_file logic_resolver_modified  --save_path results_translated_search_resolve/prompts_modified --model_name $LLM_MODEL --batch_num 1 --negation False --search_round 10 --max_new_tokens $MAX_NEW_TOKENS

In [ ]:
#!python search_resolve.py --data_path results_translated_decomposition/v2/prompts_refine --dataset_name ProntoQA --sample_pct 10 --prompts_folder manual_prompts_translated --prompts_file logic_resolver_modified  --save_path results_translated_search_resolve/prompts_modified --model_name $LLM_MODEL --batch_num 1 --negation True --search_round 10 --max_new_tokens $MAX_NEW_TOKENS 

In [ ]:
#!python evaluate.py --dataset_name ProntoQA --save_path results_translated_search_resolve/v2/prompts_modified --model_name $LLM_MODEL

ID: ProntoQA_328, GT: A, Ans1: A, Ans2: C
ID: ProntoQA_13, GT: A, Ans1: A, Ans2: C
ID: ProntoQA_380, GT: A, Ans1: A, Ans2: C
ID: ProntoQA_58, GT: B, Ans1: C, Ans2: B
ID: ProntoQA_141, GT: B, Ans1: C, Ans2: C
Total instances: 5
Accuracy: 80.00%
Error id:  ['ProntoQA_141']
Correct id:  ['ProntoQA_328', 'ProntoQA_13', 'ProntoQA_380', 'ProntoQA_58']


In [ ]:
# Logic Resolver with refined prompts
#!python search_resolve.py --data_path results_translated_decomposition/v2/prompts_refine --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file logic_resolver_refine --save_path results_translated_search_resolve/prompts_refine --model_name $LLM_MODEL --batch_num 1 --negation False --search_round 10 --max_new_tokens $MAX_NEW_TOKENS

In [ ]:
# Logic Resolver with refined prompts
#!python search_resolve.py --data_path results_translated_decomposition/v2/prompts_refine --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file logic_resolver_refine  --save_path results_translated_search_resolve/prompts_refine --model_name $LLM_MODEL --batch_num 1 --negation True --search_round 10 --max_new_tokens $MAX_NEW_TOKENS

In [5]:
!python evaluate.py --dataset_name ProntoQA --save_path results_translated_search_resolve/v3/prompts_refine --model_name $LLM_MODEL

ID: ProntoQA_415, GT: B, Ans1: C, Ans2: B
ID: ProntoQA_80, GT: A, Ans1: A, Ans2: C
ID: ProntoQA_13, GT: A, Ans1: A, Ans2: C
ID: ProntoQA_491, GT: B, Ans1: C, Ans2: B
ID: ProntoQA_309, GT: B, Ans1: A, Ans2: B
ID: ProntoQA_259, GT: B, Ans1: A, Ans2: C
ID: ProntoQA_45, GT: A, Ans1: A, Ans2: C
ID: ProntoQA_302, GT: A, Ans1: A, Ans2: C
ID: ProntoQA_53, GT: A, Ans1: A, Ans2: C
ID: ProntoQA_446, GT: B, Ans1: C, Ans2: B
ID: ProntoQA_360, GT: B, Ans1: C, Ans2: B
ID: ProntoQA_14, GT: B, Ans1: C, Ans2: B
ID: ProntoQA_112, GT: B, Ans1: C, Ans2: B
ID: ProntoQA_280, GT: B, Ans1: C, Ans2: B
ID: ProntoQA_195, GT: B, Ans1: C, Ans2: B
ID: ProntoQA_82, GT: A, Ans1: A, Ans2: C
ID: ProntoQA_328, GT: A, Ans1: A, Ans2: C
ID: ProntoQA_72, GT: A, Ans1: A, Ans2: C
ID: ProntoQA_215, GT: B, Ans1: C, Ans2: C
ID: ProntoQA_175, GT: B, Ans1: C, Ans2: B
ID: ProntoQA_413, GT: B, Ans1: C, Ans2: C
ID: ProntoQA_173, GT: B, Ans1: C, Ans2: B
ID: ProntoQA_126, GT: B, Ans1: C, Ans2: B
ID: ProntoQA_367, GT: B, Ans1: C, Ans2: B